# User-based Collaborative Filtering

Core idea:
"If Alice and Bob have rated movies similarly in the past, then we can recommend to Alice the movies that Bob liked (but Alice hasn’t seen yet)."

How it works:

Find users similar to the target user (e.g., Alice).

Look at what those similar users liked.

Recommend those movies to Alice.

Example:
If Alice and Bob both liked Inception and The Matrix, and Bob also liked Interstellar, then Interstellar might be recommended to Alice.

In [10]:
import pandas as pd
import os
base_path = r'C:\Users\Sara\Documents\DS Masters\Semester 2\Solution Engineering\Python\movies-database\ml-25m'

User-based Collaborative Filtering

In [37]:
# Load ratings and movie info
movies = pd.read_csv(os.path.join(base_path, 'movies.csv'))
ratings = pd.read_csv(os.path.join(base_path, 'ratings.csv'))

# Merge to get movie titles (optional, for display)
ratings_merged = ratings.merge(movies[['movieId', 'title']], on='movieId', how='left')

# Drop missing values (just in case)
ratings_merged.dropna(subset=['userId', 'movieId', 'rating'], inplace=True)



In [12]:
# Check the structure
ratings_merged.head()

,userId,movieId,rating,timestamp,title
0,1,296,5.0,1147880044,Pulp Fiction (1994)
1,1,306,3.5,1147868817,Three Colors: Red (Trois couleurs: Rouge) (1994)
2,1,307,5.0,1147868828,Three Colors: Blue (Trois couleurs: Bleu) (1993)
3,1,665,5.0,1147878820,Underground (1995)
4,1,899,3.5,1147868510,Singin' in the Rain (1952)


In [13]:
len(ratings_merged)

25000095

Option 1: Filter to “Active” Users and/or “Popular” Movies

We do this, because the full dataset is too computationally expensive for personal laptops.

In [14]:
# Keep users with at least 500 ratings
user_counts = ratings_merged['userId'].value_counts()
active_users = user_counts[user_counts >= 500].index

# Keep movies with at least 1000 ratings
movie_counts = ratings_merged['movieId'].value_counts()
popular_movies = movie_counts[movie_counts >= 1000].index

# Filter the DataFrame
filtered = ratings_merged[
    ratings_merged['userId'].isin(active_users) &
    ratings_merged['movieId'].isin(popular_movies)
]

In [15]:
len(filtered)

7127698

In [16]:
# Assuming 'filtered' is your cleaned dataset with userId, movieId, rating, and title
user_item_matrix = filtered.pivot_table(index='userId', columns='movieId', values='rating')

# Mean-center and fill NaNs with 0 (for cosine similarity)
user_item_centered = user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0).fillna(0)

# Save for later use in prediction
user_ids = user_item_centered.index.tolist()
movie_ids = user_item_centered.columns.tolist()


In [17]:
user_item_centered 

movieId,1,2,3,4,5,6,7,8,9,10,...,188301,189203,189333,189713,192385,192389,192803,194448,195159,201773
userId,,,,,,,,,,,,,,,,,,,,,
3,0.302050,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
12,0.663313,-1.336687,-1.336687,0.000000,0.000000,0.000000,-0.336687,0.0,0.0,-0.336687,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
72,0.000000,-1.371912,0.000000,0.000000,0.000000,0.628088,0.628088,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
80,0.000000,-1.561441,0.000000,0.000000,0.000000,2.438559,0.000000,0.0,0.0,2.438559,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
120,0.748718,0.000000,0.000000,0.000000,0.000000,0.000000,-0.251282,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,0.406332,-0.093668,0.000000,-0.093668,-1.093668,0.000000,0.000000,0.0,0.0,-0.093668,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
162495,0.091423,0.091423,0.591423,0.000000,-0.908577,0.000000,0.000000,0.0,0.0,1.091423,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0
162508,1.294654,0.000000,0.000000,0.000000,0.000000,-0.205346,0.000000,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0


In [ ]:
from annoy import AnnoyIndex

# Number of movies (features per user)
f = len(movie_ids)
annoy_index = AnnoyIndex(f, metric='angular')  # angular ≈ cosine

user_id_map = {}               # userId -> index in annoy
reverse_user_id_map = {}       # index in annoy -> userId

for i, user_id in enumerate(user_ids):
    vector = user_item_centered.loc[user_id].values
    annoy_index.add_item(i, vector)
    user_id_map[user_id] = i
    reverse_user_id_map[i] = user_id

# Build with 10 trees (balance speed/accuracy)
annoy_index.build(n_trees=10) # will deliver randomized results. Optionally we can save the index

True

In [19]:
def predict_rating_annoy(user_id, movie_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, k=30, min_raters=3):
    if user_id not in user_id_map or movie_id not in user_item_matrix.columns:
        return None

    u_idx = user_id_map[user_id]
    neighbors = annoy_index.get_nns_by_item(u_idx, k, include_distances=False)
    neighbor_ids = [reverse_user_id_map[i] for i in neighbors if reverse_user_id_map[i] != user_id]

    ratings = []
    for neighbor_id in neighbor_ids:
        rating = user_item_matrix.loc[neighbor_id, movie_id]
        if not np.isnan(rating):
            ratings.append(rating)

    if len(ratings) < min_raters:
        return user_item_matrix.loc[user_id].mean()  # fallback

    return np.mean(ratings)

In [20]:
def recommend_top_n_annoy(user_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, movies_df, n=5, k_neighbors=30):
    if user_id not in user_item_matrix.index:
        return []

    unrated_movies = user_item_matrix.columns[user_item_matrix.loc[user_id].isna()]
    predictions = []

    for movie_id in unrated_movies:
        pred = predict_rating_annoy(user_id, movie_id, user_item_matrix, annoy_index, user_id_map, reverse_user_id_map, k=k_neighbors)
        if pred is not None:
            predictions.append((movie_id, pred))

    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    return [(mid, movies_df.loc[movies_df.movieId == mid, 'title'].values[0], score) for mid, score in top_n]

In [21]:
import numpy as np

example_user = user_item_matrix.index[0]
example_user1 = user_item_matrix.index[1]
example_user2 = user_item_matrix.index[2]

top_recs = recommend_top_n_annoy(
    user_id=example_user,
    user_item_matrix=user_item_matrix,
    annoy_index=annoy_index,
    user_id_map=user_id_map,
    reverse_user_id_map=reverse_user_id_map,
    movies_df=filtered,
    n=5
)

top_recs1 = recommend_top_n_annoy(
    user_id=example_user1,
    user_item_matrix=user_item_matrix,
    annoy_index=annoy_index,
    user_id_map=user_id_map,
    reverse_user_id_map=reverse_user_id_map,
    movies_df=filtered,
    n=5
)
top_recs2 = recommend_top_n_annoy(
    user_id=example_user2,
    user_item_matrix=user_item_matrix,
    annoy_index=annoy_index,
    user_id_map=user_id_map,
    reverse_user_id_map=reverse_user_id_map,
    movies_df=filtered,
    n=5
)

# Print recommendations
print(f"\nTop 5 Recommendations for User {example_user}:")
for movie_id, title, score in top_recs:
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")

print(f"\nTop 5 Recommendations for User {example_user1}:")
for movie_id, title, score in top_recs1:
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")

print(f"\nTop 5 Recommendations for User {example_user2}:")
for movie_id, title, score in top_recs2:
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")


Top 5 Recommendations for User 3:
Fanny and Alexander (Fanny och Alexander) (1982) (Movie ID: 2068) — Predicted Rating: 4.67
Persona (1966) (Movie ID: 7327) — Predicted Rating: 4.67
12 Angry Men (1957) (Movie ID: 1203) — Predicted Rating: 4.50
My Neighbor Totoro (Tonari no Totoro) (1988) (Movie ID: 5971) — Predicted Rating: 4.50
Howl's Moving Castle (Hauru no ugoku shiro) (2004) (Movie ID: 31658) — Predicted Rating: 4.50

Top 5 Recommendations for User 12:
Breaking the Waves (1996) (Movie ID: 1354) — Predicted Rating: 4.75
Blade Runner (1982) (Movie ID: 541) — Predicted Rating: 4.67
400 Blows, The (Les quatre cents coups) (1959) (Movie ID: 2731) — Predicted Rating: 4.67
Stranger Than Paradise (1984) (Movie ID: 3925) — Predicted Rating: 4.67
Passion of Joan of Arc, The (Passion de Jeanne d'Arc, La) (1928) (Movie ID: 6985) — Predicted Rating: 4.67

Top 5 Recommendations for User 72:
Grand Day Out with Wallace and Gromit, A (1989) (Movie ID: 1223) — Predicted Rating: 4.81
Wallace & Gromi

Evaluation

In [22]:
from sklearn.model_selection import train_test_split

# Split the original filtered ratings
train_df, test_df = train_test_split(filtered, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split

# Split your filtered data
train_df, test_df = train_test_split(filtered, test_size=0.2, random_state=42)

# Only keep users from the Annoy index
active_user_ids = user_item_matrix.index
test_sample = test_df[test_df['userId'].isin(active_user_ids)].sample(n=10, random_state=42)


In [ ]:
test_sample

,userId,movieId,rating,timestamp,title
21094462,137111,1584,3.0,1018638796,Contact (1997)
21265013,138182,107348,3.0,1436916869,Anchorman 2: The Legend Continues (2013)
1277964,8619,7891,3.5,1118103920,"Last Man on Earth, The (Ultimo uomo della Terr..."
846402,5707,3495,1.5,1126260864,Roadside Prophets (1992)
5345694,34692,89118,4.0,1550188574,"Skin I Live In, The (La piel que habito) (2011)"
...,...,...,...,...,...
20290607,131898,1094,3.5,1185161951,"Crying Game, The (1992)"
23303175,151273,1054,2.0,976425218,Get on the Bus (1996)
7549783,49000,82767,3.5,1339635965,Rabbit Hole (2010)
2021325,13447,8610,4.0,1098125419,All of Me (1984)


In [ ]:
from sklearn.metrics import root_mean_squared_error

true_ratings = []
predicted_ratings = []

for _, row in test_sample.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    true_rating = row['rating']
    
    pred = predict_rating_annoy(
        user_id, movie_id,
        user_item_matrix=user_item_matrix,
        annoy_index=annoy_index,
        user_id_map=user_id_map,
        reverse_user_id_map=reverse_user_id_map,
        k=30
    )
    
    if pred is not None:
        true_ratings.append(true_rating)
        predicted_ratings.append(pred)

# Final RMSE
rmse = root_mean_squared_error(true_ratings, predicted_ratings)
print(f"✅ RMSE on 100 samples: {rmse:.4f}")

✅ RMSE on 100 samples: 0.8242


Option 2: Use a Sparse Matrix Instead of Full pivot_table

In [ ]:
from scipy.sparse import csr_matrix

# Reindex for compatibility
user_mapper = {uid: i for i, uid in enumerate(ratings_merged['userId'].unique())}
movie_mapper = {mid: i for i, mid in enumerate(ratings_merged['movieId'].unique())}

ratings_merged['user_index'] = ratings_merged['userId'].map(user_mapper)
ratings_merged['movie_index'] = ratings_merged['movieId'].map(movie_mapper)

# Create sparse matrix
rows = ratings_merged['user_index']
cols = ratings_merged['movie_index']
data = ratings_merged['rating']

sparse_user_item = csr_matrix((data, (rows, cols)))

SCIKIT-SURPRISE

In [23]:
# Sample 2,000 unique users from the filtered dataset
sampled_users = filtered['userId'].drop_duplicates().sample(n=2000, random_state=42)
sampled = filtered[filtered['userId'].isin(sampled_users)]


In [24]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Create a Surprise dataset from the sample
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(sampled[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


In [25]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [26]:
predictions = algo.test(testset)
rmse(predictions)

RMSE: 0.9187


0.9186541593271833

In [27]:
from collections import defaultdict

def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))
    for uid in top_n:
        top_n[uid] = sorted(top_n[uid], key=lambda x: x[1], reverse=True)[:n]
    return top_n

top_n = get_top_n(predictions, n=5)

In [28]:
user_id = list(top_n.keys())[0]

print(f"\nTop 5 Recommendations for User {user_id}:")
for movie_id, score in top_n[user_id]:
    title = sampled.loc[sampled.movieId == movie_id, 'title'].drop_duplicates().values[0]
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")


Top 5 Recommendations for User 12319:
Godfather: Part II, The (1974) (Movie ID: 1221) — Predicted Rating: 4.58
Matrix, The (1999) (Movie ID: 2571) — Predicted Rating: 4.57
Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001) (Movie ID: 4973) — Predicted Rating: 4.55
Seven (a.k.a. Se7en) (1995) (Movie ID: 47) — Predicted Rating: 4.43
Saving Private Ryan (1998) (Movie ID: 2028) — Predicted Rating: 4.39


In [ ]:
from surprise import Dataset, Reader, KNNBasic
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from collections import defaultdict

# 2. Load your filtered DataFrame
# Assumes 'filtered' contains: userId, movieId, rating, title
ratings = filtered[['userId', 'movieId', 'rating']].copy()

# 3. Prepare data for Surprise
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings, reader)

# 4. Split into training and testing sets
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# 5. Build the User-Based CF model
sim_options = {
    'name': 'cosine',
    'user_based': True  # Set to False for item-based
}
algo = KNNBasic(sim_options=sim_options)
algo.fit(trainset)

# 6. Evaluate on test set
predictions = algo.test(testset)
rmse(predictions)

In [29]:
import numpy
print(numpy.__version__)

1.26.4


Faiss

In [134]:
# Load ratings and movie info
movies = pd.read_csv(os.path.join(base_path, 'movies.csv'))
ratings = pd.read_csv(os.path.join(base_path, 'ratings.csv'))

# Merge to get movie titles (optional, for display)
ratings_merged = ratings.merge(movies[['movieId', 'title']], on='movieId', how='left')

# Drop missing values (just in case)
ratings_merged.dropna(subset=['userId', 'movieId', 'rating'], inplace=True)

# Keep users with at least 500 ratings
user_counts = ratings_merged['userId'].value_counts()
active_users = user_counts[user_counts >= 500].index

# Filter the DataFrame to keep only active users
filtered = ratings_merged[ratings_merged['userId'].isin(active_users)]

# Keep movies with at least 500 ratings
movie_counts = ratings_merged['movieId'].value_counts()
popular_movies = movie_counts[movie_counts >= 200].index

# Filter the dataset to include only the popular movies
filtered = filtered[filtered['movieId'].isin(popular_movies)]

# Check the filtered dataset size
print(f"Filtered dataset shape: {filtered.shape}")


Filtered dataset shape: (8305815, 5)


In [135]:
filtered["movieId"].nunique(), filtered["userId"].nunique()

(7984, 9713)

In [ ]:
import faiss
from sklearn.preprocessing import normalize

# Create user-item matrix
user_item_matrix = filtered.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
user_vectors = normalize(user_item_matrix.values.astype('float32'))

# Build FAISS index (cosine similarity ≈ dot product on normalized vectors)
index = faiss.IndexFlatIP(user_vectors.shape[1])  # IP = Inner Product
index.add(user_vectors)

In [138]:
index

<faiss.swigfaiss_avx2.IndexFlatIP; proxy of <Swig Object of type 'faiss::IndexFlatIP *' at 0x000002A764A53B10> >

In [139]:
target_user_id = 3
target_idx = user_item_matrix.index.get_loc(target_user_id)
D, I = index.search(np.expand_dims(user_vectors[target_idx], axis=0), k=5)

# Output: indices of similar users + similarity scores
similar_users = user_item_matrix.index[I[0]]
print(similar_users)

Index([3, 124561, 149968, 159639, 91040], dtype='int64', name='userId')


In [160]:
def normalize_user_item_matrix(user_item_matrix):
    # Normalize the user-item matrix by subtracting each user's average rating
    # Calculate the mean of each user's ratings (ignoring NaN values)
    user_means = user_item_matrix.mean(axis=1)
    
    # Subtract the user's mean rating from each of their ratings
    user_item_matrix_normalized = user_item_matrix.sub(user_means, axis=0)
    
    return user_item_matrix_normalized

In [166]:
user_item_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,201588,201646,201749,201773,202103,202429,202439,203222,204698,205383
userId,,,,,,,,,,,,,,,,,,,,,
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,4.0,2.0,2.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
72,0.0,2.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
80,0.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
120,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162484,3.5,3.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162495,3.0,3.0,3.5,0.0,2.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
162508,4.5,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def recommend_top_n_faiss(user_id, user_item_matrix, index, user_vectors, movies_df, n=5, k=5):
    """
    Recommend top-N movies for a given user using FAISS-based nearest neighbors.

    Parameters:
    - user_id: ID of the target user.
    - user_item_matrix: DataFrame containing user-item ratings.
    - index: FAISS index built from user vectors.
    - user_vectors: Numpy array of user vectors.
    - movies_df: DataFrame containing movie information with 'movieId' and 'title' columns.
    - n: Number of top recommendations to return.
    - k: Number of nearest neighbors to consider.

    Returns:
    - List of tuples: (movieId, title, predicted_rating)
    """
    if user_id not in user_item_matrix.index:
        return []

    # Normalize the user-item matrix by subtracting the user's average rating
    user_item_matrix_normalized = normalize_user_item_matrix(user_item_matrix)

    # Get the index of the target user
    target_idx = user_item_matrix.index.get_loc(user_id)

    # Search for k+1 nearest neighbors (including the user themselves)
    D, I = index.search(np.expand_dims(user_vectors[target_idx], axis=0), k=k+1)

    # Retrieve neighbor indices and distances
    neighbor_indices = I[0]
    distances = D[0]

    # Map indices to user IDs
    neighbor_ids = user_item_matrix.index[neighbor_indices]

    # Exclude the target user from neighbors
    mask = neighbor_ids != user_id
    neighbor_ids = neighbor_ids[mask]
    distances = distances[mask]

    # Identify movies not yet rated by the target user (those rated as 0)
    unrated_movies = user_item_matrix.columns[user_item_matrix.loc[user_id] == 0]
    
    if unrated_movies.empty:
        print(f"User {user_id} has already rated all movies. No unrated movies left to recommend.")
        return []  # Return empty list if no unrated movies are left
    
    predictions = []

    for movie_id in unrated_movies:
        # Retrieve ratings from neighbors for the current movie (from the normalized matrix)
        neighbor_ratings = user_item_matrix_normalized.loc[neighbor_ids, movie_id]

        # Drop ratings that are NaN (neighbors who haven't rated the movie)
        valid_ratings = neighbor_ratings.dropna()

        if valid_ratings.empty:
            continue  # Skip if no neighbor has rated the movie

        # Compute similarities (inverse of distances)
        similarities = 1 / (distances[:len(valid_ratings)] + 1e-10)  # Add epsilon to avoid division by zero
        
        # Normalize similarities to ensure they are not too skewed
        if np.sum(similarities) > 0:
            similarities = similarities / np.sum(similarities)

        # Compute weighted average of neighbor ratings
        weighted_sum = np.dot(valid_ratings.values, similarities)
        sum_of_weights = np.sum(similarities)

        # Calculate predicted rating in the normalized scale
        predicted_rating_normalized = weighted_sum / sum_of_weights

        # Get the user's average rating to "de-normalize" the prediction
        user_avg_rating = user_item_matrix.loc[user_id].mean()

        # Add the user's average back to the predicted rating
        predicted_rating = predicted_rating_normalized + user_avg_rating
        
        # Clamp predictions between 1 and 5
        predicted_rating = min(5, max(1, predicted_rating))

        predictions.append((movie_id, predicted_rating))

    if not predictions:
        print("No predictions made.")
        return []

    # Sort predictions by predicted rating in descending order
    top_n = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

    # Retrieve movie titles
    recommendations = []
    for movie_id, score in top_n:
        title = movies_df.loc[movies_df['movieId'] == movie_id, 'title'].values
        title = title[0] if len(title) > 0 else "Unknown Title"
        recommendations.append((movie_id, title, score))

    return recommendations


In [ ]:
user_id = 12  # Replace with the target user's ID
recommendations = recommend_top_n_faiss(user_id, user_item_matrix, index, user_vectors, movies, n=5, k=5)

print(f"Top 5 Recommendations for User {user_id}:")
for movie_id, title, score in recommendations:
    print(f"{title} (Movie ID: {movie_id}) — Predicted Rating: {score:.2f}")

Top 5 Recommendations for User 12:
Graduate, The (1967) (Movie ID: 1247) — Predicted Rating: 4.22
Chinatown (1974) (Movie ID: 1252) — Predicted Rating: 4.01
Do the Right Thing (1989) (Movie ID: 3424) — Predicted Rating: 3.98
Sense and Sensibility (1995) (Movie ID: 17) — Predicted Rating: 3.90
Taxi Driver (1976) (Movie ID: 111) — Predicted Rating: 3.88


In [167]:
# Get the movies that the user has rated
rated_movies = user_item_matrix.loc[user_id][user_item_matrix.loc[user_id] != 0]  # Non-zero ratings

print(f"Movies rated by User {user_id}:")
for movie_id, rating in rated_movies.items():
    title = movies.loc[movies['movieId'] == movie_id, 'title'].values
    title = title[0] if len(title) > 0 else "Unknown Title"
    print(f"{title} (Movie ID: {movie_id}) — Rating: {rating:.2f}")

Movies rated by User 12:
Toy Story (1995) (Movie ID: 1) — Rating: 4.00
Jumanji (1995) (Movie ID: 2) — Rating: 2.00
Grumpier Old Men (1995) (Movie ID: 3) — Rating: 2.00
Sabrina (1995) (Movie ID: 7) — Rating: 3.00
GoldenEye (1995) (Movie ID: 10) — Rating: 3.00
Casino (1995) (Movie ID: 16) — Rating: 5.00
Four Rooms (1995) (Movie ID: 18) — Rating: 4.00
Money Train (1995) (Movie ID: 20) — Rating: 1.00
Get Shorty (1995) (Movie ID: 21) — Rating: 2.00
Copycat (1995) (Movie ID: 22) — Rating: 3.00
Leaving Las Vegas (1995) (Movie ID: 25) — Rating: 3.00
Now and Then (1995) (Movie ID: 27) — Rating: 2.00
City of Lost Children, The (Cité des enfants perdus, La) (1995) (Movie ID: 29) — Rating: 4.00
Dangerous Minds (1995) (Movie ID: 31) — Rating: 2.00
Twelve Monkeys (a.k.a. 12 Monkeys) (1995) (Movie ID: 32) — Rating: 2.00
Dead Man Walking (1995) (Movie ID: 36) — Rating: 4.00
Clueless (1995) (Movie ID: 39) — Rating: 4.00
To Die For (1995) (Movie ID: 45) — Rating: 2.00
Seven (a.k.a. Se7en) (1995) (Movie 